In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
import numpy as np
import quandl
import os
import warnings
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from collections import defaultdict
from graham import *

warnings.filterwarnings('ignore')
quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')

In [16]:
list_sp500 = get_sp500_symbols()
len(list_sp500)

505

In [49]:
start_date = datetime(2008, 1, 1)
end_date = datetime(2008, 1, 5)


def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

        
list_dfs = []
i = 1
for b in batch(list_sp500, 100):
    print(f'Batch {i}: from {b[0]} to {b[-1]}')
    df1 = quandl.get_table('SHARADAR/SEP', ticker=b, date={'gte': start_date, 'lte': end_date}, paginate=True,
                        qopts={"columns":['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']})
    
    df2 = quandl.get_table('SHARADAR/DAILY', ticker=b, date={'gte': start_date, 'lte': end_date}, paginate=True,
                            qopts={"columns":['ticker', 'date', 'pe', 'pb']})
    result = pd.merge(df1, df2, on=['ticker', 'date'])
    result.sort_values(['ticker', 'date'], inplace=True)
    
    list_dfs.append(result)
    i += 1

final_df = pd.concat(list_dfs, axis=0)

Batch 1: from MMM to CELG
Batch 2: from CNC to FLIR
Batch 3: from FLS to LYB
Batch 4: from MTB to REGN
Batch 5: from RF to XLNX
Batch 6: from XYL to ZTS


In [52]:
final_df.shape

(1332, 9)

In [53]:
1332/444

3.0

In [47]:
b = ['KO', 'GE', 'AAPL']
df1 = quandl.get_table('SHARADAR/SEP', ticker=b, date={'gte': start_date, 'lte': end_date}, paginate=True,
                        qopts={"columns":['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']})
    
df2 = quandl.get_table('SHARADAR/DAILY', ticker=b, date={'gte': start_date, 'lte': end_date}, paginate=True,
                        qopts={"columns":['ticker', 'date', 'pe', 'pb']})
result = pd.merge(df1, df2, on=['ticker', 'date'])
result.sort_values(['ticker', 'date'], inplace=True)
result

,ticker,date,open,high,low,close,volume,pe,pb
20,AAPL,2008-01-02,28.467,28.609,27.507,27.834,269794700.0,48.8,11.7
19,AAPL,2008-01-03,27.916,28.199,27.527,27.847,210516600.0,48.8,11.7
18,AAPL,2008-01-04,27.350,27.571,25.556,25.721,363958000.0,45.1,10.8
17,AAPL,2008-01-07,25.893,26.229,24.319,25.377,518048300.0,44.5,10.7
16,AAPL,2008-01-08,25.734,26.066,24.400,24.464,380954000.0,42.9,10.3
15,AAPL,2008-01-09,24.471,25.643,24.043,25.629,453470500.0,44.9,10.8
14,AAPL,2008-01-10,25.369,25.857,25.059,25.431,370743800.0,44.6,10.7
13,GE,2008-01-02,37.100,37.450,36.550,36.760,38914400.0,16.9,3.3
12,GE,2008-01-03,36.800,37.220,36.700,36.800,26976200.0,16.9,3.3
11,GE,2008-01-04,36.530,36.670,35.980,36.040,40730900.0,16.5,3.2
